# Summarizing whit LLM

### En este notebook se implementa una solución para el problema número 5 del Hackathon de NataSquad.
#### -Crear un agente que reciba un fichero BibTex y devuelva un texto con los temas que se abordan en los documentos que recoge el .bib
#### -Interactuar con el agente con el fin de obtener más información respecto a los artículos y autores. Por ejemplo: Obtener el nombre del autor con mayor índice h, o el artículo más relevante, o el link de descarga de un (o todos los) artículo(s) (siempre que se tenga acceso abierto), etc.


#### Para trabajar, se creó una colección en Zotero con citas descargadas directamente de Google Scholar y de IEEE Xplorer. Se aclara ya que los gestores bibliográficos soportan otro tipo de referencias, como por ejemplo sitios web. Esta solución solo se probo con citas referidas a artículos, libros y similares.

## Importar fichero BibTex y extraer los datos más significativos

### Se utiliza la biblioteca _pybtex_ para parsear los datos del .bib 

In [1]:
from pybtex.database import parse_file

#La función get_value es para manejar errores ocasionados por la no presencia de algún valor necesario. Por ejemplo: Las referencias
#obtenidas de Google Scholar no añaden el abstract ,en IEEE Xplorer si dan la opción.
def get_value(data, key):
    try:
        value = data[key]
        return value
    except KeyError as error:
        return None

#La función get_fst_author es para obtener el nombre del autor principal. Se obtiene de los metadatos del fichero .bib        
def get_fst_author(data):
    name = {}
    name['name'] = data.first_names
    name['last'] = data.last_names
    return name

path_to_file = 'bib_natasquad/bib_natasquad.bib'
bib_data = parse_file(path_to_file)
bib = []

#El diccionario entry_data recoge los key:values fundamentales de una referencia bibliográfica para dar solución al problema planteado
#Los valores que se les asigna None es porque no están presentes en el fichero .bib y/o se pueden 
#obtener con mayor facilidad con la API de OpenAlex

for _, entry in bib_data.entries.items():
    entry_data = {
        'type': entry.type,
        'title': entry.fields['title'],
        'title_openalex': None,
        'journal': get_value(entry.fields,'journal'),
        'year': get_value(entry.fields, 'year'),
        'abstract': get_value(entry.fields,'abstract'),
        'authors': get_value(entry.persons, 'author'),
        'name_first_author': None,
        'openalex_id_first_author': None,
        'relevance_first_author': None,
        'h_index_first_author': None,
        'i10_index_first_author': None,
        'doi': get_value(entry.fields,'doi'),
        'is_open_access': False,
        'download_url': '',
        'language': None,
        'institution': None,
        'country_code': None,
        'cited_count': None,
        'relevance_score': None 
    }
    bib.append(entry_data)

## Una vez obtenidos los datos del fichero .bib se procede a completar los valores faltantes del diccionario _entry_data_

### Se usa la API REST gratuita de OpenAlex _https://openalex.org_

In [2]:
#OpenAlex permite 100,000 consultas por dia por usuario. Las consultas realizadas son GET request y la respuesta es en formato JSON
import requests    

 #Esta función elimina los '{' '}' que aparecen en algunos títulos obtenidos del fichero .bib. En caso que aparezcan otros caracteres no
#deseados habrá que aumentar la funcionalidad para eliminarlos. En la colección estudiada sólo aparecen '{' '}' 
def clear_title(title):
    title = title.replace('{','')
    title = title.replace('}','')
    return title
    
#La función check_doi es para detectar la presencia del doi en la consulta realizada a OpenAlex. Existen referencias a documentos 
#que no presentan el doi
def check_doi(res_openalex):
    if res_openalex['doi'] != None:
        return True
    else:
        return False

#OpenAlex entre los valores que devuelve no se encuentra el abstract pero si un índice invertido del abstract.
#Esta función retorna el abstract en formato str a partir del abstract_inverted_index
def get_abstarct(indice):
    ks = list(indice.keys())
    vs = list(indice.values())
    a = []
    for i in range(len(indice)):
        for j in vs[i]:
            a.insert(j, ks[i])
    abstract = ' '.join(a)
    return abstract

filters = []

#OpenAlex permite  agregar filtros a las consultas y concatenar varios filtros.
#Las líneas de código que siguen se encargan de hacer consultas a la API con el fin de completar los valores faltantes en el diccionario 
#entry_data para cada referencia presente en el fichero BibTex

url_req = "https://api.openalex.org/works?filter=title.search:"

#Hacer un recorrido por cada referencia
for i in range(len(bib)):

    #Primero se obtienen datos del autor principal para, además de completar entry_data, realizar una búsqueda del artículo lo mas precisa 
    # posible con filtros del id del autor registrado en OpenAlex
    fs = bib[i]['authors'][0].first_names[0]
    sd = bib[i]['authors'][0].last_names[0]
    fs_author_name = fs + ' ' + sd   

    url_author = 'https://api.openalex.org/authors?search=' + fs_author_name
    
    res_author = requests.get(url_author).json()
    author_json = res_author['results'][0]
    
    bib[i]['name_first_author'] = author_json['display_name']
    bib[i]['openalex_id_first_author'] = author_json['id'][21:]
    bib[i]['relevance_first_author'] = get_value(author_json,'relevance_score')
    bib[i]['h_index_first_author'] = get_value(author_json['summary_stats'],'h_index')
    bib[i]['i10_index_first_author'] = get_value(author_json['summary_stats'],'i10_index')

    #La primera consulta para obtener información de los trabajos presenta filtros como:
    #1-Texto presente en el título
    #2-ID del autor principal en la base de datos de OpenAlex
    #3-Si presenta doi
    #El objetivo es obtener la respuesta más precisa 
    url = url_req + bib[i]['title'] + ',' + 'authorships.author.id:' +  bib[i]['openalex_id_first_author']+ ',' + 'has_doi:true'
    res_openalex = requests.get(url).json()
    
    #Si el resultado de la consulta es 0 se limpia el título y solo se filtra por <texto que aparece en el título>
    if res_openalex['meta']['count'] == 0:
        bib[i]['title'] =  clear_title(bib[i]['title'])
        url = url_req + bib[i]['title'] 
        res_openalex = requests.get(url).json()

    #Escoger qué resultado de la API utilizar. Si retorna más de 1 trabajo comparar el título de los trabajos retornados con el título
    #del trabajo que se está analizando
    if res_openalex['meta']['count'] > 1:
        for j in range(res_openalex['meta']['count']):
            if res_openalex['results'][j]['title'] == bib[i]['title']:
                res = res_openalex['results'][j]
                break
            else:
                res = res_openalex['results'][0]
                continue
    elif res_openalex['meta']['count'] == 1:
        res = res_openalex['results'][0]

    bib[i]['title_openalex'] = res['title']
    
    #Con el resultado adecuado proporcionado por OpenAlex se procede a completar los valores faltantes correspondietes a la referencia
    if bib[i]['doi'] == None:
        if check_doi(res):
            bib[i]['doi'] = res['doi'][16:]

    if bib[i]['abstract'] == None:
        if res['abstract_inverted_index'] != None:
            bib[i]['abstract'] = get_abstarct(res['abstract_inverted_index'])

    bib[i]['is_open_access'] = res['open_access']['is_oa']
    if(bib[i]['is_open_access'] == True):
        bib[i]['download_url'] = res['open_access']['oa_url']

    bib[i]['language'] = res['language']    

    if(len(res['authorships'][0]['institutions']) != 0):
        bib[i]['institution'] = res['authorships'][0]['institutions'][0]['display_name']
        bib[i]['country_code'] = res['authorships'][0]['institutions'][0]['country_code']

    bib[i]['cited_count'] = res['cited_by_count']

    bib[i]['relevance_score'] = res['relevance_score']

## Crear DataFrame a partir de una lista de diccionarios con la forma key:value de _entry_data_ 
### Se lleva a este formato porque se utilizará un framework (LlamaIndex) que permite conectar este tipo de datos con LLMs como los modelos GPT3.5 de OpenAI
  

### Se utiliza la biblioteca pandas

In [3]:
import pandas as pd

df = pd.DataFrame(bib)
df

,type,title,title_openalex,journal,year,abstract,authors,name_first_author,openalex_id_first_author,relevance_first_author,h_index_first_author,i10_index_first_author,doi,is_open_access,download_url,language,institution,country_code,cited_count,relevance_score
0,article,Artificial neural networks enabled by nanophot...,Artificial neural networks enabled by nanophot...,Light: Science \& Applications,2019,Abstract The growing demands of brain science ...,"[Zhang, Qiming, Yu, Haoyi, Barbiero, Martina, ...",Qiming Zhang,A5019450682,10920.4960,58,214,10.1038/s41377-019-0151-0,True,https://doi.org/10.1038/s41377-019-0151-0,en,RMIT University,AU,147,1493.363200
1,article,On the binding problem in artificial neural ne...,On the Binding Problem in Artificial Neural Ne...,arXiv preprint arXiv:2012.05208,2020,Contemporary neural networks still fall short ...,"[Greff, Klaus, Van Steenkiste, Sjoerd, Schmidh...",Klaus Greff,A5082808731,6868.8545,17,20,10.48550/arxiv.2012.05208,True,http://arxiv.org/pdf/2012.05208,en,None,None,3,176.200270
2,article,Comprehensive review of artificial neural netw...,Comprehensive Review of Artificial Neural Netw...,IEEE access,2019,The era of artificial neural network (ANN) beg...,"[Abiodun, Oludare Isaac, Jantan, Aman, Omolara...",Oludare Isaac Abiodun,A5050410057,1750.1735,8,8,10.1109/access.2019.2945545,True,https://doi.org/10.1109/access.2019.2945545,en,Universiti Sains Malaysia,MY,194,1916.792800
3,article,Artificial neural networks: A tutorial,Artificial neural networks: a tutorial,Computer,1996,Artificial neural nets (ANNs) are massively pa...,"[Jain, Anil K., Mao, Jianchang, Mohiuddin, K. ...",Anil K. Jain,A5007029060,12709.4910,165,822,10.1109/2.485891,False,,en,Michigan State University,US,2923,5839.766600
4,book,Fundamentals of artificial neural networks,Fundamentals of artificial neural networks,None,1995,None,"[Hassoun, Mohamad H.]",Mohamad Rani Hassoun,A5053102906,NaN,0,0,10.1016/0021-9290(96)82229-9,False,,en,University of Michigan–Ann Arbor,US,1,100.760635
5,article,Introduction to the artificial neural networks,Introduction to the Artificial Neural Networks,Artificial Neural Networks: Methodological Adv...,2011,An Artificial Neural Network (ANN) is a mathem...,"[Krenker, Andrej, Bešter, Janez, Kos, Andrej]",Andrej Krenker,A5088808065,1359.0453,2,2,10.5772/15751,True,https://www.intechopen.com/citation-pdf-url/14881,en,None,None,118,997.785000
6,inproceedings,Extracting training data from large language m...,Extracting Training Data from Large Language M...,None,2021,It has become common to publish large (billion...,"[Carlini, Nicholas, Tramer, Florian, Wallace, ...",Nicholas Carlini,A5034257647,9699.5990,38,55,10.48550/arxiv.2012.07805,True,http://arxiv.org/pdf/2012.07805,en,None,None,1,113.158195
7,article,Large language models in machine translation,Large Language Models in Machine Translation,None,2007,"Systems, methods, and computer program product...","[Brants, Thorsten, Popat, Ashok C., Xu, Peng, ...",Thorsten Brants,A5076738898,6706.2010,26,40,None,False,,en,Google (United States),US,326,1940.008500
8,article,Deep learning,Deep learning,nature,2015,None,"[LeCun, Yann, Bengio, Yoshua, Hinton, Geoffrey]",Yann LeCun,A5001226970,41925.7150,94,233,10.1038/nature14539,False,,en,Meta (United States),US,48137,11073.689000
9,book,Deep learning,Deep Learning with Differential Privacy,None,2016,Machine learning techniques based on neural ne...,"[Goodfellow, Ian, Bengio, Yoshua, Courville, A...",Ian J. Goodfellow,A5004271128,8755.2720,66,114,10.1145/2976749.2978318,True,http://arxiv.org/pdf/1607.00133,en,Google (United States),US,2517,2232.040500


## Como se va hacer uso de las APIs que proporciona OpenAI de manera directa, usando la biblioteca openai, y también a través del framework LlamaIndex es necesario una API KEY

In [4]:
import os
import openai

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
openai.api_key = OPENAI_API_KEY


#### Para generar el resumen se parte del abstract de cada referencia (en caso que lo presente). Se usa el módulo __*Completion*__, el abstract se utiliza como __*promt*__ de entrada al modelo _text-davinci-003_. Al __*promt*__ se le adiciona una "instrucción" para que el modelo ejecute sobre el abstract. Se le pone como límite un máximo de 50 tokens en la respuesta para moderar el consumo y se pone una __*temperature*__ igual a 0 para obtener respuestas poco aleatorias


In [5]:
out_openai = []
a_null = df['abstract'].isnull()
for i in a_null.keys():
    if a_null.values[i]:
        out_openai.append('Without Abstract')
    else:
        output = openai.Completion.create(
                        model="text-davinci-003",
                        prompt=df['abstract'][i] + '\n\n Summarize in one sentence the central idea of the text:',
                        max_tokens=50,
                        temperature=0
                    )
        out_openai.append(output['choices'][0]['text'][2:])

In [6]:
for i in range(len(out_openai)):
    print("Title: {}".format(df['title_openalex'][i]))
    print("Summary: {}".format(out_openai[i]))
    print('=================================================================================\n')


Title: Artificial neural networks enabled by nanophotonics
Summary: Nanophotonics has enabled the development of neural networks that can mimic the structural, functional, and biological features of human networks.

Title: On the Binding Problem in Artificial Neural Networks
Summary: The central idea of this paper is to propose a unifying framework for neural networks to enable them to acquire a compositional understanding of the world through symbolic processing and grounded representations.

Title: Comprehensive Review of Artificial Neural Network Applications to Pattern Recognition
Summary: The text discusses the progress made in Artificial Neural Network (ANN) applications in Pattern Recognition (PR) and the need for further research to address existing challenges and enable more successes.

Title: Artificial neural networks: a tutorial
Summary: The development of Artificial Neural Nets (ANNs) is motivated by the desire to replicate the biological neuron in a computational model, a


## Retrieval Augmented Generation (RAG) es un paradigma para aumentar los LLM con datos personalizados. 
### <pre>__Generalmente consiste en dos etapas:__</pre><pre>1. indexado: prepara la base de conocimiento, y</pre> <pre>2. consulta: recuperar el contexto relevante del conocimiento para ayudar al LLM a responder una pregunta</pre> 
<img src="rag.jpg" alt="drawing" style="width:600px;"/>

##### https://gpt-index.readthedocs.io/en/latest/getting_started/concepts.html

## Para la interacción con los datos se usa el framework LlamaIndex, que permite conectar datos estructurados, semi-estructurados y no estructurados a LLM como los GPT de OpenAI. LlamaIndex a traves de los modelos de OpenAI convierte las consultas realizadas en lenguaje natural a consultas del DataFrame para obtener la respuesta deseada (En este caso de uso, tiene muchas otras funcionalidades). 

### Se utiliza un Query Engine con el objetivo de crear un asistente de pregunta-respuesta. Específicamente se utiliza el Pandas Query Engine, el cual permite pasarle un DataFrame y hacerle consultas en lenguaje natural 

In [7]:
from llama_index.query_engine import PandasQueryEngine

query_engine = PandasQueryEngine(df=df)

### Realizar un ciclo infinito esperando una entrada en lenguaje natural por parte del usuario y se imprime la respuesta generada basada en los datos del fichero .bib 

In [ ]:
from IPython.display import Markdown, display
while(True):
    display(Markdown(f"<b>{'Ask me about the BibTex file:'}</b>"))
    user_search = input()
    res = query_engine.query(user_search)
    display(Markdown(f"><b>{res.response}</b>"))

<b>Ask me about the BibTex file:</b>

 cuantos trabajos hay


len(df)


><b>13</b>

<b>Ask me about the BibTex file:</b>

...

<b>Ask me about the BibTex file:</b>

 que articulos puedo descargar


df[df['download_url'] != ''][['title', 'download_url']]


><b>                                                title  \
0   Artificial neural networks enabled by nanophot...   
1   On the binding problem in artificial neural ne...   
2   Comprehensive review of artificial neural netw...   
5      Introduction to the artificial neural networks   
6   Extracting training data from large language m...   
9                                       Deep learning   
10           Deep learning in agriculture: {A} survey   
11  LSTM-CNN Architecture for Human Activity Recog...   

                                         download_url  
0           https://doi.org/10.1038/s41377-019-0151-0  
1                     http://arxiv.org/pdf/2012.05208  
2         https://doi.org/10.1109/access.2019.2945545  
5   https://www.intechopen.com/citation-pdf-url/14881  
6                     http://arxiv.org/pdf/2012.07805  
9                     http://arxiv.org/pdf/1607.00133  
10                    http://arxiv.org/pdf/1807.11809  
11        https://doi.org/10.1109/access.2020.2982225  </b>

<b>Ask me about the BibTex file:</b>